# Image Classification:-
### The data consists of images pertaining to 10 categories. The task is to train a convolutional neural net in keras for classifying these images.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['10_categories-1551435405057']


### Setting Path

In [2]:
import os
PATH = os.getcwd()
print(PATH)

/kaggle/working


In [3]:
DATA_PATH = "../input/10_categories-1551435405057/10_categories"

In [4]:
print(os.listdir(DATA_PATH))
data_dir_list = os.listdir(DATA_PATH)

['BACKGROUND_Google', 'Leopards', 'watch', 'grand_piano', 'Motorbikes', 'car_side', 'airplanes', 'bonsai', 'Faces', 'Faces_easy']


### Required variables declaration and initialization

In [5]:
img_rows=224
img_cols=224
num_channel=3

num_epoch=50
batch_size=64

img_data_list=[]
classes_names_list=[]
target_column_list=[]

### Reading Data

In [6]:
import cv2

for dataset in data_dir_list:
    classes_names_list.append(dataset)
    print ('Loading images from {} folder\n'.format(dataset)) 
    img_list=os.listdir(DATA_PATH+'/'+ dataset)
    for img in img_list:
        input_img=cv2.imread(DATA_PATH + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(img_rows, img_cols))
        img_data_list.append(input_img_resize)
        target_column_list.append(dataset)

Loading images from BACKGROUND_Google folder

Loading images from Leopards folder

Loading images from watch folder

Loading images from grand_piano folder

Loading images from Motorbikes folder

Loading images from car_side folder

Loading images from airplanes folder

Loading images from bonsai folder

Loading images from Faces folder

Loading images from Faces_easy folder



In [7]:
num_classes = len(classes_names_list)
num_images =  len(img_data_list)
print(num_classes)
print(num_images)

10
3725


In [8]:
import numpy as np

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

In [9]:
print (img_data.shape)

(3725, 224, 224, 3)


In [10]:
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape
print(num_of_samples)
print(input_shape)

3725
(224, 224, 3)


In [11]:
from sklearn.preprocessing import LabelEncoder

classes = np.ones((num_of_samples,), dtype='int64')
labelEncoder = LabelEncoder()
classes = labelEncoder.fit_transform(target_column_list)
print(classes)

[0 0 0 ... 2 2 2]


In [12]:
np.unique(classes)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [13]:
from tensorflow.keras.utils import to_categorical

classes = to_categorical(classes, num_classes)

In [14]:
classes.shape

(3725, 10)

### Splitting into Train and Test

In [15]:
from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=2)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [17]:
y_test.shape

(745, 10)

### Basic Image Classification Model

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [19]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 64)        0         
__________

In [22]:
hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))

Train on 2980 samples, validate on 745 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
2980/2980 [==============================] - 14s 5ms/sample - loss: 2.3158 - acc: 0.2829 - val_loss: 1.9269 - val_acc: 0.4295
Epoch 2/50
2980/2980 [==============================] - 8s 3ms/sample - loss: 1.4916 - acc: 0.4745 - val_loss: 1.1486 - val_acc: 0.5960
Epoch 3/50
2980/2980 [==============================] - 9s 3ms/sample - loss: 1.2156 - acc: 0.5762 - val_loss: 1.2321 - val_acc: 0.6685
Epoch 4/50
2980/2980 [==============================] - 8s 3ms/sample - loss: 1.0822 - acc: 0.6275 - val_loss: 0.9388 - val_acc: 0.6792
Epoch 5/50
2980/2980 [==============================] - 9s 3ms/sample - loss: 1.0357 - acc: 0.6419 - val_loss: 0.8624 - val_acc: 0.7181
Epoch 6/50
2980/2980 [==============================] - 8s 3ms/sample - loss: 0.9234 - acc: 0.6772 - val_loss: 0.8074 - val_acc: 0.7409
Epoch 7/50
2980/2980 [==============================] - 9s 3ms/sample - loss: 0.8749 - a

In [23]:
score = model.evaluate(X_test, y_test, batch_size=batch_size)

print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

745/745 [==============================] - 1s 2ms/sample - loss: 0.3314 - acc: 0.9275
Test Loss: 0.33136088982124456
Test Accuracy: 0.92751676


In [24]:
test_image = X_test[0:1]
print (test_image.shape)

(1, 224, 224, 3)


In [25]:
print(model.predict(test_image))
print(model.predict_classes(test_image))
print(y_test[0:1])

[[4.3611371e-13 1.6008456e-05 9.9998403e-01 4.1685248e-22 2.8149419e-23
  4.8994080e-15 6.0854377e-25 7.2678095e-15 1.1892427e-14 7.6944219e-16]]
[2]
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [26]:
from sklearn.metrics import confusion_matrix

Y_pred = model.predict(X_test)
print(Y_pred)

[[4.3611206e-13 1.6008395e-05 9.9998403e-01 ... 7.2677824e-15
  1.1892382e-14 7.6943928e-16]
 [5.0654131e-24 1.3622348e-10 1.0000000e+00 ... 1.0579345e-26
  1.7630201e-22 1.2672635e-27]
 [7.4155480e-01 1.0442579e-03 1.0061817e-04 ... 3.6107798e-03
  1.3762071e-04 1.6007343e-03]
 ...
 [2.1022420e-02 1.1194828e-03 2.1322336e-05 ... 9.7543210e-01
  3.9064806e-04 3.3562948e-04]
 [3.7258590e-20 1.3557415e-29 2.1125039e-25 ... 2.9865573e-20
  5.8999165e-27 6.2571115e-20]
 [1.5980925e-02 4.9443088e-07 1.3185932e-11 ... 2.8638656e-06
  5.1099079e-08 2.8596558e-05]]


In [27]:
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)

[2 2 0 0 2 2 3 5 6 1 5 5 3 0 7 5 1 9 1 5 4 4 4 4 5 4 4 0 4 5 5 4 5 5 4 5 5
 4 4 4 4 2 1 5 5 4 0 4 4 9 0 5 5 4 4 1 2 0 9 1 9 5 0 4 1 4 5 5 5 0 2 2 5 4
 4 4 2 4 9 8 0 0 7 0 2 2 7 3 2 3 8 4 5 7 0 5 4 4 9 5 1 3 4 9 1 5 5 4 5 4 5
 0 5 1 1 4 0 5 1 0 2 1 5 2 4 0 5 5 4 4 0 4 4 1 4 2 1 6 1 8 0 5 0 6 3 0 0 4
 1 6 4 0 5 4 5 5 5 0 5 4 5 4 5 7 5 6 7 4 9 4 3 3 4 4 4 5 5 1 4 1 9 5 0 5 0
 5 1 2 5 2 0 4 1 3 0 4 0 5 2 4 3 4 4 1 4 2 1 0 1 4 2 4 5 4 7 5 0 5 5 1 7 0
 0 3 6 2 0 7 5 5 0 4 0 0 5 7 7 0 1 3 4 0 5 0 2 4 5 5 5 4 9 5 2 2 0 4 3 4 1
 1 9 4 4 3 0 9 2 0 0 7 5 4 0 4 2 4 5 4 0 1 4 5 4 8 5 5 5 0 0 1 1 5 5 1 1 5
 4 1 3 0 5 3 4 1 0 5 0 5 1 0 2 9 0 0 4 2 5 1 0 5 1 4 4 9 0 9 3 5 0 4 9 3 5
 4 7 2 6 7 0 3 5 0 7 4 9 7 4 5 0 2 4 0 6 0 5 4 1 0 3 3 2 1 3 1 4 0 5 4 0 9
 3 1 4 0 5 0 1 4 1 4 5 6 9 0 0 0 5 9 8 3 0 2 2 6 0 4 0 2 0 2 9 3 1 3 1 4 4
 2 5 5 0 0 3 0 5 6 2 2 0 5 1 4 4 4 0 4 2 1 2 4 5 4 1 0 0 5 5 5 1 4 5 5 3 4
 5 0 4 9 2 9 6 1 4 9 3 4 4 1 0 5 2 0 3 3 4 5 7 2 3 4 3 5 9 0 2 4 8 7 3 5 5
 5 0 2 0 7 5 0 4 2 3 3 9 

In [28]:
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

[[ 95   0   0   1   3   2   1   1   0   1]
 [  4  81   1   0   0   0   1   0   0   0]
 [  1   0  81   0   0   0   0   0   0   0]
 [  0   0   0  45   0   0   0   0   0   0]
 [  0   0   0   0 157   1   0   0   0   0]
 [  3   0   0   0   3 153   1   0   0   0]
 [ 10   0   0   0   0   0  12   1   0   1]
 [  0   0   0   0   1   0   0  23   0   0]
 [  3   0   0   0   0   0   0   0   7   0]
 [  9   0   0   1   1   2   0   1   0  37]]


### Image Augmentation

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(
    rotation_range=20,
    shear_range=0.5, 
    zoom_range=0.4, 
    rescale=1./255,
    vertical_flip=True, 
    validation_split=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True) 

In [30]:
TRN_AUGMENTED = os.path.join(PATH , 'Trn_Augmented_Images')
TST_AUGMENTED = os.path.join(PATH , 'Tst_Augmented_Images')

In [31]:
ftrain_generator = data_gen.flow(
        X_train,
        y_train,
        batch_size=batch_size, 
        shuffle=True,  
        subset="training")

In [32]:
ftest_generator = data_gen.flow(
        X_test,
        y_test,
        batch_size=batch_size, 
        shuffle=True,  
        subset="validation")

In [33]:
num_epoch=50
model.fit_generator(ftrain_generator, epochs = num_epoch, validation_data=ftest_generator)

Epoch 1/50
38/38 [==============================] - 25s 664ms/step - loss: 2.3792 - acc: 0.1019 - val_loss: 2.2944 - val_acc: 0.1208
Epoch 2/50
38/38 [==============================] - 25s 645ms/step - loss: 2.2158 - acc: 0.1812 - val_loss: 2.1380 - val_acc: 0.2416
Epoch 3/50
38/38 [==============================] - 25s 649ms/step - loss: 2.1397 - acc: 0.2047 - val_loss: 2.1036 - val_acc: 0.2416
Epoch 4/50
38/38 [==============================] - 24s 621ms/step - loss: 2.1289 - acc: 0.2018 - val_loss: 2.0688 - val_acc: 0.2416
Epoch 5/50
38/38 [==============================] - 24s 624ms/step - loss: 2.1291 - acc: 0.2051 - val_loss: 2.0558 - val_acc: 0.2416
Epoch 6/50
38/38 [==============================] - 25s 666ms/step - loss: 2.1137 - acc: 0.2152 - val_loss: 2.0281 - val_acc: 0.2416
Epoch 7/50
38/38 [==============================] - 25s 646ms/step - loss: 2.1071 - acc: 0.2223 - val_loss: 2.0228 - val_acc: 0.2416
Epoch 8/50
38/38 [==============================] - 24s 644ms/step - 

Epoch 39/50
38/38 [==============================] - 25s 666ms/step - loss: 2.1050 - acc: 0.2009 - val_loss: 2.0062 - val_acc: 0.2416
Epoch 40/50
38/38 [==============================] - 24s 619ms/step - loss: 2.1043 - acc: 0.2198 - val_loss: 2.0063 - val_acc: 0.2416
Epoch 41/50
38/38 [==============================] - 24s 621ms/step - loss: 2.0916 - acc: 0.2194 - val_loss: 2.0087 - val_acc: 0.2416
Epoch 42/50
38/38 [==============================] - 25s 654ms/step - loss: 2.0950 - acc: 0.2173 - val_loss: 2.0078 - val_acc: 0.2416
Epoch 43/50
38/38 [==============================] - 24s 621ms/step - loss: 2.0963 - acc: 0.2244 - val_loss: 1.9915 - val_acc: 0.2416
Epoch 44/50
38/38 [==============================] - 24s 621ms/step - loss: 2.0944 - acc: 0.2143 - val_loss: 1.9896 - val_acc: 0.2416
Epoch 45/50
38/38 [==============================] - 25s 657ms/step - loss: 2.0903 - acc: 0.2081 - val_loss: 2.0358 - val_acc: 0.2416
Epoch 46/50
38/38 [==============================] - 25s 671ms

In [35]:
Y_pred = model.predict(X_test)
print(Y_pred)

[[3.02342705e-05 2.34097727e-02 9.76512551e-01 ... 1.28207134e-07
  1.09881055e-08 3.68969552e-07]
 [1.20394546e-07 4.83344169e-03 9.95162964e-01 ... 6.83333815e-12
  1.75988148e-11 1.91886104e-10]
 [1.36979163e-01 1.34361327e-01 1.31216004e-01 ... 4.17281240e-02
  3.37003246e-02 7.49236494e-02]
 ...
 [1.67613417e-01 1.09480217e-01 1.00978822e-01 ... 1.67003404e-02
  1.56054487e-02 4.67821620e-02]
 [2.66273048e-09 6.19622279e-12 2.32975572e-09 ... 4.03864550e-13
  8.88800192e-18 1.13201074e-10]
 [9.68017504e-02 1.56470924e-03 1.54043199e-04 ... 4.39621590e-06
  5.60943192e-07 2.57343549e-04]]


In [36]:
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)

[2 2 4 4 2 2 3 5 4 1 5 5 3 4 0 5 1 9 1 5 4 4 4 4 5 4 4 0 4 5 5 4 5 5 4 5 5
 4 4 4 4 2 1 5 5 4 0 4 4 9 4 5 5 4 4 1 1 0 9 1 9 5 0 4 5 4 5 5 5 0 2 2 5 4
 4 4 2 4 4 0 4 0 4 4 2 2 4 3 2 3 4 4 5 4 0 5 4 4 4 5 0 3 4 0 0 5 5 4 5 4 5
 0 5 1 1 4 4 5 1 3 2 1 5 2 4 4 5 5 4 4 3 4 4 1 4 2 1 4 1 4 0 5 0 0 3 4 0 4
 1 0 4 4 5 4 5 5 5 0 4 4 4 4 4 5 5 0 4 4 9 4 3 3 4 4 4 5 5 1 4 1 9 5 0 5 0
 5 1 2 5 2 0 4 1 3 0 4 0 5 4 4 3 4 4 1 4 2 1 4 5 4 2 4 5 4 1 5 0 5 5 1 1 4
 0 3 4 2 0 0 5 5 0 4 0 0 5 4 4 0 1 3 4 0 4 0 2 4 5 5 5 4 4 5 2 2 4 4 3 4 1
 0 9 4 4 3 0 5 2 0 0 0 5 4 0 4 2 4 5 4 4 1 4 5 4 0 5 5 4 0 0 1 1 5 5 1 9 5
 4 1 3 4 5 3 4 1 0 5 0 4 1 0 2 4 0 0 4 2 5 1 4 4 1 4 4 9 0 9 3 5 0 4 4 3 5
 4 4 2 4 4 4 3 5 5 4 4 9 0 4 5 0 2 4 0 5 0 5 4 1 0 3 3 2 4 3 1 4 9 5 4 0 9
 4 1 4 0 5 4 1 4 1 4 5 0 9 0 0 4 5 9 0 3 1 2 2 5 0 4 4 2 0 2 9 3 1 3 1 4 4
 2 5 5 5 4 3 4 5 5 2 2 4 5 1 4 4 4 0 4 2 1 2 4 4 4 1 0 0 5 5 5 1 4 5 5 3 4
 5 4 4 9 2 9 0 1 4 9 3 4 4 1 0 5 2 4 3 3 4 5 0 2 3 4 3 5 5 5 2 4 4 5 3 5 5
 5 4 2 0 4 5 0 4 2 3 3 9 

In [37]:
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

[[ 62   1   0   4  33   4   0   0   0   0]
 [  5  72   2   0   3   3   0   0   0   2]
 [  1   1  79   0   1   0   0   0   0   0]
 [  0   0   0  44   1   0   0   0   0   0]
 [  0   0   0   0 157   1   0   0   0   0]
 [  1   0   0   0  16 143   0   0   0   0]
 [ 13   0   0   0   8   3   0   0   0   0]
 [  6   2   0   0  14   2   0   0   0   0]
 [  6   0   0   0   4   0   0   0   0   0]
 [  9   0   0   0  16   3   0   0   0  23]]


# Transaffer Learning

In [38]:
from tensorflow.keras.layers import Input, Dense

In [39]:
image_input = Input(shape=(img_rows, img_cols, num_channel))

In [40]:
from tensorflow.keras.applications.vgg16 import VGG16

model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

553467904/553467096 [==============================] - 13s 0us/step


In [41]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [42]:
last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)

In [43]:
from tensorflow.keras.models import Model

custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [44]:
for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False

In [45]:
custom_vgg_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [46]:
custom_vgg_model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))

Train on 2980 samples, validate on 745 samples
Epoch 1/50
2980/2980 [==============================] - 13s 4ms/sample - loss: 1.7686 - acc: 0.4886 - val_loss: 1.4031 - val_acc: 0.5074
Epoch 2/50
2980/2980 [==============================] - 10s 3ms/sample - loss: 0.7555 - acc: 0.7594 - val_loss: 0.7541 - val_acc: 0.6765
Epoch 3/50
2980/2980 [==============================] - 10s 3ms/sample - loss: 0.4820 - acc: 0.8443 - val_loss: 0.3280 - val_acc: 0.9114
Epoch 4/50
2980/2980 [==============================] - 10s 3ms/sample - loss: 0.3398 - acc: 0.8919 - val_loss: 0.6279 - val_acc: 0.7342
Epoch 5/50
2980/2980 [==============================] - 10s 3ms/sample - loss: 0.2791 - acc: 0.9141 - val_loss: 0.3741 - val_acc: 0.8510
Epoch 6/50
2980/2980 [==============================] - 10s 3ms/sample - loss: 0.2273 - acc: 0.9295 - val_loss: 0.4746 - val_acc: 0.8201
Epoch 7/50
2980/2980 [==============================] - 10s 3ms/sample - loss: 0.1894 - acc: 0.9460 - val_loss: 0.1628 - val_acc: 0

In [47]:
(loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

745/745 [==============================] - 2s 3ms/sample - loss: 0.1510 - acc: 0.9490
[INFO] loss=0.1510, accuracy: 94.8993%


In [48]:
Y_train_pred = custom_vgg_model.predict(X_test)

In [49]:
y_train_pred = np.argmax(Y_train_pred, axis=1)
print(y_train_pred)

[2 2 6 0 2 2 3 5 6 1 5 5 3 9 7 5 1 9 2 5 4 4 4 4 5 4 4 8 4 5 5 4 5 5 4 5 5
 4 4 4 4 2 1 5 5 4 0 4 4 9 8 5 5 4 4 1 2 0 9 1 9 5 0 4 1 4 5 5 5 9 2 2 5 4
 4 4 2 4 9 8 6 6 7 0 2 2 7 3 2 3 8 4 5 7 6 5 4 4 6 5 1 3 4 9 1 5 5 4 5 5 5
 0 5 1 1 4 6 5 1 0 2 1 5 2 4 0 5 5 4 4 0 4 4 1 4 2 1 6 1 8 0 5 6 6 3 0 5 4
 1 0 4 0 5 4 5 5 5 0 5 4 5 4 5 7 5 6 7 4 9 4 3 3 4 4 4 5 5 1 4 1 9 5 9 8 0
 6 1 2 5 2 0 4 1 3 8 4 0 5 2 4 3 6 4 1 4 2 1 1 1 4 2 4 5 4 7 5 6 5 5 1 7 0
 0 3 6 2 6 7 5 5 4 4 0 0 5 7 7 0 1 3 4 0 9 8 2 4 5 5 5 4 9 5 2 2 2 4 3 9 1
 1 9 4 4 3 0 6 2 6 1 7 5 4 1 4 2 4 5 4 9 1 4 5 4 8 5 5 5 0 0 1 1 5 5 1 1 5
 4 1 3 6 5 3 4 1 0 5 6 0 1 0 2 9 0 5 4 2 5 1 6 5 1 4 4 9 9 9 3 5 0 4 9 3 5
 4 7 6 6 6 0 3 5 0 7 4 9 7 4 5 8 2 4 0 6 8 5 4 1 0 3 3 2 1 3 1 4 1 5 4 0 9
 9 1 5 4 5 1 1 4 1 4 5 6 9 0 6 9 5 9 8 3 0 2 2 6 9 4 0 2 9 2 9 3 1 3 1 4 4
 2 5 5 9 0 3 6 5 1 2 2 0 5 1 4 4 4 2 6 2 1 2 4 5 4 1 0 8 5 4 5 1 4 5 5 3 4
 5 0 4 9 2 9 6 1 4 9 0 4 7 1 8 5 2 0 3 3 4 5 7 2 3 4 3 5 9 5 2 4 8 7 3 5 5
 5 0 2 8 7 5 6 4 2 3 3 9 

In [50]:
print(confusion_matrix(np.argmax(y_test, axis=1), y_train_pred))

[[ 70   1   1   0   2   1  17   0   7   5]
 [  0  86   1   0   0   0   0   0   0   0]
 [  0   0  81   0   0   0   1   0   0   0]
 [  0   0   0  45   0   0   0   0   0   0]
 [  0   0   0   0 158   0   0   0   0   0]
 [  0   0   0   0   0 158   0   0   2   0]
 [  0   0   0   0   0   0  24   0   0   0]
 [  0   0   0   0   0   0   0  24   0   0]
 [  0   0   0   0   0   0   0   0  10   0]
 [  0   0   0   0   0   0   0   0   0  51]]
